In [1]:
import torch
import pennylane as qml
from time import time
from tqdm import tqdm
import numpy as np
import matplotlib as plt
from circuit_model_II import QuantumCircuit, FullQuantumModel
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import os

In [2]:
num_qubits = 8
num_layers = 5
model = FullQuantumModel(num_qubits, num_layers)

# Dataset preparation

In [3]:
# Download MNIST and prepare transforms
mnist_train = datasets.MNIST(root='./data', train=True, download=True,
                             transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
# Filter for zeros and ones
data = []
targets = []
for image, label in mnist_train:
    if label in [0, 1]:
        data.append(image.squeeze())
        targets.append(label)

data = torch.stack(data)
targets = torch.tensor(targets)
# Select 1024 zeros and 1024 ones for speed
zeros_indices = (targets == 0)
ones_indices = (targets == 1)

zeros = data[zeros_indices]
ones = data[ones_indices]

# take a subsample of the dataset for simplicity
zeros = zeros[:1024]
ones = ones[:1024]

#normalize between 0 and 1
zeros_max = torch.max(zeros.reshape(-1, 16*16), dim = 1)
zeros_min = torch.min(zeros.reshape(-1, 16*16), dim = 1)
ones_max = torch.max(ones.reshape(-1, 16*16), dim = 1)
ones_min = torch.min(ones.reshape(-1, 16*16), dim = 1)

def normalize(imgs):
  maxes, _ = torch.max(imgs.reshape(-1, 16*16), dim = 1)
  mins, _ = torch.min(imgs.reshape(-1, 16*16), dim = 1)

  mins = mins.unsqueeze(1).unsqueeze(2)
  maxes = maxes.unsqueeze(1).unsqueeze(2)

  return (imgs-mins)/(maxes-mins)

zeros = normalize(zeros)
ones = normalize(ones)

# assert images have min 0 and max 1 within an error of 1e-5
assert torch.allclose(zeros.min(), torch.tensor(0., dtype = torch.float32), atol=1e-5)
assert torch.allclose(zeros.max(), torch.tensor(1., dtype = torch.float32), atol=1e-5)
assert torch.allclose(ones.min(), torch.tensor(0., dtype = torch.float32), atol=1e-5)
assert torch.allclose(ones.max(), torch.tensor(1., dtype = torch.float32), atol=1e-5)

# concatenate the two datasets
zeros = zeros.flatten(start_dim = 1)
ones = ones.flatten(start_dim = 1)
dataset = torch.cat((zeros, ones), dim = 0)

# add labels
labels = torch.cat((torch.zeros((zeros.shape[0], 1)), torch.ones((ones.shape[0], 1))), dim = 0).squeeze()

# build dataloader
dataset = torch.utils.data.TensorDataset(dataset, labels)

BATCH_SIZE = 32

dataloader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last = True)

In [4]:
dataloader.batch_size

32

# Model training

In [11]:
model.fit(dataloader=dataloader, learning_rate=0.01, epochs=10)

Epoch 1/10:   3%|▎         | 2/64 [00:00<00:03, 15.99it/s, accuracy=1, loss=0.302]    

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])


Epoch 1/10:   9%|▉         | 6/64 [00:00<00:03, 16.82it/s, accuracy=1, loss=0.318]    

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])


Epoch 1/10:  16%|█▌        | 10/64 [00:00<00:03, 16.43it/s, accuracy=1, loss=0.302]

...

Epoch 10/10:  92%|█████████▏| 59/64 [00:03<00:00, 19.98it/s, accuracy=1, loss=0.334]    

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])


Epoch 10/10: 100%|██████████| 64/64 [00:03<00:00, 19.14it/s, accuracy=0.969, loss=0.289]

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
Time per epoch:  3.3440439701080322
Epoch:  9 Loss:  0.2892584502696991
Accuracy:  0.96875
--------------------------------------------------------------------------


(3.222240614891052,
 [0.29758214950561523,
  0.30170902609825134,
  0.32587286829948425,
  0.31278491020202637,
  0.3051925599575043,
  0.3128332197666168,
  0.29793015122413635,
  0.2923735976219177,
  0.31433677673339844,
  0.2892584502696991])

In [12]:
for i in model.parameters(): 
    print(i)
    break

Parameter containing:
tensor([[ 1.4929,  0.9107,  1.0181],
        [ 0.9554, -0.5808, -0.2439],
        [ 0.5648, -0.2558,  1.7541],
        [ 1.4368,  1.1390, -0.0140],
        [ 0.8342, -0.1917,  1.8964],
        [ 0.5762,  0.1116,  1.4495],
        [ 0.7821,  0.0411,  0.5196],
        [ 0.5933,  0.8271,  1.3459]], requires_grad=True)


In [10]:
model.unfreeze_layers([1,2])
for i in model.parameters(): 
    print(i)
    break

Parameter containing:
tensor([[ 1.3629,  1.0201,  0.8152],
        [ 0.8423, -0.4657, -0.1720],
        [ 0.8119, -0.0925,  1.7360],
        [ 1.4267,  0.9895,  0.0350],
        [ 0.8196, -0.1364,  1.5823],
        [ 0.4226,  0.2861,  1.4179],
        [ 0.9527,  0.1969,  0.5144],
        [ 0.3481,  0.5851,  1.3831]], requires_grad=True)


NameError: name 'norm' is not defined